Scoring of ambulances based on distance,ETA,severity of the disease detected, facilities like cardiac assistance,oxygen supply and ventialtion requirement and selecting the ambulance based on this score

In [3]:

import pandas as pd
import numpy as np
import folium
import geopy.distance
import requests
import time
df=pd.read_csv('ambulance.csv')
df=pd.DataFrame(df)


In [ ]:

def batch_eta(locations,API_KEY,url):
    """
    Batch process ETA using OpenRouteService API.
    """
    payload = {
        "locations": locations,  # List of coordinates
        "metrics": ["duration"]  # Request duration as metric
    }
    headers = {
        "Authorization": API_KEY,
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json().get('durations', [])
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None


In [5]:
def gaussian(x, mu, sigma=1):
  return np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

In [6]:
def compute_score(row, severity):
    score = 0

    # Assign centers for each ambulance type
    centers = {
        'Non-Emergency': 2,
        'BLS': 5.5,
        'ALS': 8
    }

    # Assign the standard deviation (you can adjust this for broader or narrower curves)
    sigma = 2  # This controls how wide or narrow the Gaussian curve is

    # Compute the Gaussian score based on severity and ambulance type
    if severity >= 1 and severity <= 4:
        # Penalize non-emergency, and decrease the penalty for ALS and BLS based on Gaussian curve
        score += gaussian(severity, centers['Non-Emergency'], sigma)  # Non-Emergency gets high score for severity 1-4
        score -= gaussian(severity, centers['ALS'], sigma)  # ALS gets a penalty
        score -= gaussian(severity, centers['BLS'], sigma)  # BLS gets some penalty

    elif severity >= 4 and severity <= 7:
        # BLS should get higher score here
        score += gaussian(severity, centers['BLS'], sigma)
        score -= gaussian(severity, centers['Non-Emergency'], sigma)
        score -= gaussian(severity, centers['ALS'], sigma)

    elif severity >= 7 and severity <= 10:
        # ALS should get the highest score for severity 7-10
        score += gaussian(severity, centers['ALS'], sigma)
        score -= gaussian(severity, centers['Non-Emergency'], sigma)
        score -= gaussian(severity, centers['BLS'], sigma)

    return score


In [ ]:
def score_generation(lat,lon,API_KEY,url):
  df['ETA']=0
  locations = df[[ 'Longitude','Latitude']].values.tolist()
  origin = [lon,lat]
  all_coordinates = [[loc, origin] for loc in locations]
  etas = []
  for batch in all_coordinates:
   duration_matrix = batch_eta(batch,API_KEY,url)
   if duration_matrix:
      etas.append(duration_matrix[0][1]//60)  # Append duration for each origin-to-destination
      time.sleep(2)
  df['ETA']=etas
  distance_km=[]

  for row in df.itertuples(index=False):
    distance_km.append(geopy.distance.distance([lat,lon],(row.Latitude,row.Longitude)).km)
  df['Distance']=distance_km

  df['Score'] = df.apply(lambda row: compute_score(row, severity), axis=1)
  df['Score'] = df.apply(lambda row: bonus_scoring(row,severity,cardiac,oxygen,ventilation), axis=1)
  return df[df['Score']==df['Score'].max()]


In [ ]:
def bonus_scoring(row,severity,cardiac=0,oxygen=0,ventialation=0):
  score=row['Score']
  if (cardiac & row['Cardiac'] == 1):
    score += 1
  if (oxygen & row['Oxygen']) == 1:
        score += 1
  if (ventialation & row['Ventilator']) == 1:
        score += 1

  # Penalize based on distance (closer ambulances are better)
  normalized_distance = 1 / (row['Distance'] + 1)  # Adding 1 to avoid division by zero
  score += normalized_distance * 5  # Scale distance importance

  score += (1/(row['ETA']))*5
  return score

In [ ]:
# OpenRouteService API key
API_KEY = '5b3ce3597851110001cf6248eb38bd3e8a4748269c41294631f48c0e'

url = "https://api.openrouteservice.org/v2/matrix/driving-car"

lat=input("Enter the latitude of your location: ")
lon=input("Enter the longitude of your location:")
severity=input("Enter the severity of the accident: ")
oxygen=input("oxyen")
ventilation=input("ventilation")
cardiac=input("cardiac")
score_generation(lat,lon,API_KEY,url)


Vizualization of result

In [ ]:
import folium
m=folium.Map(location=[lat,lon],zoom_start=10)
for i,row in df.iterrows():
  lats=df.at[i,'Latitude']
  lons=df.at[i,'Longitude']
  amb=df.at[i,'Ambulance_Numberplate']

  folium.Marker(location=[lats,lons],popup=amb,icon=folium.Icon(color="green")).add_to(m)
folium.Marker(location=[lat,lon],popup="My Location",icon=folium.Icon(color="red")).add_to(m)
m

In [ ]:
df.to_csv('ambulance_result.csv')